In [ ]:
from load_mnist import *
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

import datetime

In [ ]:
def plot_images(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = matplotlib.cm.binary, **options)
    plt.axis("off")

# Load traditional MNIST dataset
This dataset is taken from http://yann.lecun.com/exdb/mnist/

In [ ]:
images_tr, labels_tr = load_mnist('MNIST/')
images_tst, labels_tst = load_mnist('MNIST/', 't10k')

X_tr, y_tr = images_tr, labels_tr
X_tst, y_tst = images_tst, labels_tst

In [ ]:
# show some images
plt.figure(figsize=(9,9))
example_images = np.r_[X_tr[:12000:600], X_tr[13000:30600:600], X_tr[30600:60000:590]]
plot_images(example_images, images_per_row=10)
# save_fig("more_images_plot")
plt.show()

In [ ]:
# plot images for each label
X_0 = X_tr[(y_tr == 0)]
X_1 = X_tr[(y_tr == 1)]
X_2 = X_tr[(y_tr == 2)]
X_3 = X_tr[(y_tr == 3)]
X_4 = X_tr[(y_tr == 4)]
X_5 = X_tr[(y_tr == 5)]
X_6 = X_tr[(y_tr == 6)]
X_7 = X_tr[(y_tr == 7)]
X_8 = X_tr[(y_tr == 8)]
X_9 = X_tr[(y_tr == 9)]

plt.figure(figsize=(24,24))
plt.subplot(521); plot_images(X_0[:25], images_per_row=5)
plt.subplot(522); plot_images(X_1[:25], images_per_row=5)
plt.subplot(523); plot_images(X_2[:25], images_per_row=5)
plt.subplot(524); plot_images(X_3[:25], images_per_row=5)
plt.subplot(525); plot_images(X_4[:25], images_per_row=5)
plt.subplot(526); plot_images(X_5[:25], images_per_row=5)
plt.subplot(527); plot_images(X_6[:25], images_per_row=5)
plt.subplot(528); plot_images(X_7[:25], images_per_row=5)
plt.subplot(529); plot_images(X_8[:25], images_per_row=5)
plt.subplot(5,2,10); plot_images(X_9[:25], images_per_row=5)
# save_fig("images_for_each_label")
plt.show()

## Scaler

Rescale the values of the image data into the interval [0.01, 0.99] by dividing (255 * 0.98 + 0.01), which put values between 0 and 1 but not including 0 and 1.

In [ ]:
X_tr = X_tr / 255 * 0.98 + 0.01
X_tst = X_tst / 255 * 0.98 + 0.01

## Binary Classifier

Choose 5 and 6

In [ ]:
#Select 5 & 6 from training and test
train_filter = np.where((y_tr == 5 ) | (y_tr == 6))
test_filter = np.where((y_tst == 5) | (y_tst == 6))
X_tr_b, y_tr_b = X_tr[train_filter], y_tr[train_filter]
X_tst_b, y_tst_b = X_tst[test_filter], y_tst[test_filter]

#Convert to 0 and 1
y_tr_binary = (y_tr_b == 5).astype(np.float)
y_tst_binary = (y_tst_b == 5).astype(np.float)

y_tr_binary_expand = np.expand_dims(y_tr_binary, axis=1)
y_tst_binary_expand = np.expand_dims(y_tst_binary, axis=1)

In [ ]:
def sigmoid(t):
    return np.exp(t) / (1 + np.exp(t))

In [ ]:
def calculate_loss(y, tx, w):
    p = sigmoid(tx.dot(w))
    loss = np.sum(-y * np.log(p) - (1 - y) * np.log(1 - p)) / y.shape[0]
    return loss

In [ ]:
def calculate_gradient(y, tx, w):
    p = sigmoid(tx.dot(w))
    gradient = np.dot(tx.T, (p - y)) / y.shape[0]
    return gradient

In [ ]:
def learning_by_gradient_descent(y, tx, w, tau):
    loss = calculate_loss(y, tx, w)
    gradient = calculate_gradient(y, tx, w)
    w -= tau * gradient
    return loss, w

In [ ]:
def predict_probs(tx, w):
    return sigmoid(tx.dot(w))

In [ ]:
def predict(tx, w, threshold=0.5):
    return predict_probs(tx, w) >= threshold

In [ ]:
def logistic_regression_gradient_descent_demo(y, x, max_iter, tau):
    # init parameters
    max_iter = max_iter
    threshold = 1e-8
    tau = tau #change to see the diff
    losses = []
    accuracies = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 1))

    # Start GD
    start_time = datetime.datetime.now()
    
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, tau)
        prediction = predict(tx, w, threshold=0.5).astype(np.float)
#         print(prediction)
        accuracy = (prediction == y).mean()
        tprediction = predict(np.c_[np.ones((y_tst_b.shape[0], 1)), X_tst_b], w, threshold=0.5).astype(np.float)
        taccuracy = (tprediction == y_tst_binary_expand).mean()
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}, training accuracy={a}, test accuracy={ta}"
                  .format(i=iter, l=loss, a=accuracy, ta=taccuracy))
        # converge criterion
        accuracies.append(accuracy)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    end_time = datetime.datetime.now()
    exection_time = (end_time - start_time).total_seconds()
    # Print result
    print("Binary GD: execution time={t:.3f} seconds".format(t=exection_time))
    # visualization
    #plt.plot(losses)
    plt.plot(accuracies)
    #visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_gradient_descent")
    #print("loss={l}".format(l=calculate_loss(y, tx, w)))
    
    # return for confusion matrix
    y_pred = prediction.flatten()
    y_act = y.flatten()
    return y_pred, y_act

In [ ]:
y_pred, y_act = logistic_regression_gradient_descent_demo(y_tr_binary_expand, X_tr_b, 1000, 1.4)

In [ ]:
# Visualize the Confusion Matrix
# np.set_printoptions(suppress=True) # disable scientific numbers
y_act = pd.Series(y_act, name='Actual')
y_pred = pd.Series(y_pred, name='Predicted')
conf_mat = pd.crosstab(y_act, y_pred)
# Normalized confusion matrix
conf_mat_norm = conf_mat / conf_mat.sum(axis=1)
# print(df_confusion)
# sn.heatmap(conf_mat, annot=True)

In [ ]:
labels = ['5', '6']
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
# plt.title('Binary Confusion Matrix',fontsize=16)
plt.xlabel('Predict')
plt.ylabel('Actual')

for (i, j), z in np.ndenumerate(conf_mat):
    ax.text(j, i, '{:0}'.format(z), ha='center', va='center') # for default in integer
#     ax.text(j, i, '{:0.2%}'.format(z), ha='center', va='center') # for normalized in percentage
    
plt.show()

## Newton's method

In [ ]:
def calculate_loss(y, tx, w):
    p = sigmoid(tx.dot(w))
    loss = np.sum(-y * np.log(p) - (1 - y) * np.log(1 - p)) / y.shape[0]
    return loss

In [ ]:
def predict(tx, w, threshold=0.5):
    return predict_probs(tx, w) >= threshold

In [ ]:
def calculate_hessian(y, tx, w, alpha):
    p = sigmoid(tx.dot(w))
    diag = np.diag((p * (1 - p)).flatten())
    hessian = tx.T.dot(diag).dot(tx) / y.shape[0] + alpha * np.identity(w.shape[0])
    return hessian

In [ ]:
def logistic_regression(y, tx, w, alpha):
    loss = calculate_loss(y, tx, w)
    gradient = calculate_gradient(y, tx, w)
    hessian = calculate_hessian(y, tx, w, alpha)
    return loss, gradient, hessian

In [ ]:
def inv(m):
    a, b = m.shape
    if a != b:
        raise ValueError("Only square matrices are invertible.")
    i = np.eye(a,a)
    return np.linalg.lstsq(m, i, rcond=None)[0]

In [ ]:
def learning_by_newton_method(y, tx, w, alpha):
    loss, gradient, hessian = logistic_regression(y, tx, w, alpha)
    loss += (alpha/2) * np.sum(w*w)
    gradient += alpha*w
    w -= np.linalg.solve(hessian, gradient)
    #w -= np.dot(inv(hessian), gradient)
    return loss, w

In [ ]:
def logistic_regression_newton_method_demo(y, x):
    # init parameters
    max_iter = 100
    threshold = 1e-8
    tau = 1e-7
    alpha = 0.01
    losses = []
    accuracies = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 1))

    # Start Newton's
    start_time = datetime.datetime.now()
    
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_newton_method(y, tx, w, alpha)
        # log info
        prediction = predict(tx, w, threshold=0.5).astype(np.float)
        accuracy = (prediction == y).mean()
        tprediction = predict(np.c_[np.ones((y_tst_b.shape[0], 1)), X_tst_b], w, threshold=0.5).astype(np.int)
        taccuracy = (tprediction == y_tst_binary_expand).mean()
        if iter % 1 == 0:
            print("Current iteration={i}, the loss={l}, training accuracy={a}, test accuracy={ta}"
                  .format(i=iter, l=loss, a=accuracy, ta=taccuracy))
        # converge criterion
        losses.append(loss)
        accuracies.append(accuracy)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break

    # Print result
    end_time = datetime.datetime.now()
    exection_time = (end_time - start_time).total_seconds()
    print("Newton's: execution time={t:.3f} seconds".format(t=exection_time))
    # visualization
    #plt.plot(losses)
    plt.plot(accuracies)

In [ ]:
logistic_regression_newton_method_demo(y_tr_binary_expand, X_tr_b)

## Stochastic Gradient Descent (Binary)

In [ ]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

In [ ]:
def stochastic_gradient_descent(y, tx, w, batch_size, max_iter, tau, alpha):
    """Stochastic gradient descent."""
    losses = []
    accuracies = []
    for n_iter in range(max_iter):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            gradient = calculate_gradient(y_batch, tx_batch, w)
            gradient += alpha*w
            # calculate loss
            loss = calculate_loss(y, tx, w)
            loss += (alpha/2) * np.sum(w*w)
            # update w through the stochastic gradient update
            w -= tau * gradient
            
            prediction = predict(tx, w, threshold=0.5).astype(np.float)
            accuracy = (prediction == y).mean()
            tprediction = predict(np.c_[np.ones((y_tst_b.shape[0], 1)), X_tst_b], w, threshold=0.5).astype(np.float)
            taccuracy = (tprediction == y_tst_binary_expand).mean()
            losses.append(loss)
            accuracies.append(accuracy)
        # visualization
        plt.plot(accuracies)
        print("SGD({bi}/{ti}): loss={l}, training accuracy={a}, test accuracy={ta}".format(
              bi=n_iter, ti=max_iter - 1, l=loss, a=accuracy, ta=taccuracy))
    return loss, w, accuracy, prediction

In [ ]:
def logistic_regression_stochastic_gradient_descent_demo(y, x, max_iter, tau):
    # init parameters
    max_iter = max_iter
    threshold = 1e-8
    tau = tau #change to see the diff
    alpha = 1e-7
    batch_size = 1

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 1))

    # Start SGD
    start_time = datetime.datetime.now()
    # start the logistic regression
    # get loss and update w.
    loss, w, accuracy, prediction = stochastic_gradient_descent(y, tx, w, batch_size, max_iter, tau, alpha)
    #taccuracy = getAccuracy(X_tst, y_tst, w)
    # log info
    end_time = datetime.datetime.now()
    exection_time = (end_time - start_time).total_seconds()
    # Print result
    print("SGD: execution time={t:.3f} seconds".format(t=exection_time))

    
    # return for confusion matrix
    y_pred = prediction.flatten()
    y_act = y.flatten()
    return y_pred, y_act

In [ ]:
y_pred, y_act = logistic_regression_stochastic_gradient_descent_demo(y_tr_binary_expand, X_tr_b, 1000, 0.1)

In [ ]:
# Visualize the Confusion Matrix
# np.set_printoptions(suppress=True) # disable scientific numbers
y_act = pd.Series(y_act, name='Actual')
y_pred = pd.Series(y_pred, name='Predicted')
conf_mat = pd.crosstab(y_act, y_pred)
# Normalized confusion matrix
conf_mat_norm = conf_mat / conf_mat.sum(axis=1)
# print(df_confusion)
# sn.heatmap(conf_mat, annot=True)

In [ ]:
labels = ['5', '6']
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
# plt.title('Binary Confusion Matrix',fontsize=16)
plt.xlabel('Predict')
plt.ylabel('Actual')

for (i, j), z in np.ndenumerate(conf_mat):
    ax.text(j, i, '{:0}'.format(z), ha='center', va='center') # for default in integer
#     ax.text(j, i, '{:0.2%}'.format(z), ha='center', va='center') # for normalized in percentage
    
plt.show()

## Softmax

In [ ]:
def onehotencoding(y):
    return (np.arange(np.max(y) + 1) == y[:, None]).astype(np.float)

In [ ]:
def softmax(z):
    z -= np.max(z)
    return (np.exp(z.T) / np.sum(np.exp(z), axis=1)).T

In [ ]:
def calculate_loss(y, tx, w):
    softm = softmax(tx.dot(w))
    loss = - np.sum(y * np.log(softm)) / y.shape[0]
    prediction = np.argmax(softm, axis=1)
    return softm, loss, prediction

In [ ]:
def calculate_gradient(y, tx, w):
    softm = softmax(tx.dot(w))
    return - np.dot(tx.T, (y - softm)) / y.shape[0]

In [ ]:
def learning_by_gradient_descent(y, tx, w, tau, alpha):
    y_enc = onehotencoding(y)
    softm, loss, prediction = calculate_loss(y_enc, tx, w)
    loss += (alpha/2) * np.sum(w*w)
    gradient = calculate_gradient(y_enc, tx, w)
    gradient += alpha
    w -= tau * gradient
    accuracy = np.sum(prediction == y)/ y.shape[0]
    return loss, w, accuracy, prediction

In [ ]:
def getAccuracy(x, y, w):
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    softm = softmax(tx.dot(w))
    prediction = np.argmax(softm, axis=1)
    accuracy = np.sum(prediction == y)/ y.shape[0]
    return accuracy

In [ ]:
def logistic_regression_gradient_descent_demo(y, x, max_iter, tau):
    # init parameters
    max_iter = max_iter
    threshold = 1e-8
    tau = tau #change to see the diff
    alpha = 1e-6
    losses = []
    accuracies = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 10))
    
    # Start GD
    start_time = datetime.datetime.now()
    
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w, accuracy, prediction = learning_by_gradient_descent(y, tx, w, tau, alpha)
        taccuracy = getAccuracy(X_tst, y_tst, w)
        tprediction = np.argmax(softmax(np.c_[np.ones((y_tst.shape[0], 1)), X_tst].dot(w)), axis=1)
        # log info
        if iter % 50 == 0:
            print("Current iteration={i}, loss={l}, training accuracy={a}, test accuracy={ta}"
                  .format(i=iter, l=loss, a=accuracy, ta=taccuracy))
        # converge criterion
        losses.append(loss)
        accuracies.append(accuracy)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # Print result
    end_time = datetime.datetime.now()
    exection_time = (end_time - start_time).total_seconds()
    print("GD: execution time={t:.3f} seconds".format(t=exection_time))
    # visualization
    #plt.plot(losses)
    plt.plot(accuracies)
    #visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_gradient_descent")
    #print("loss={l}".format(l=calculate_loss(onehotencoding(y), tx, w)))
    # return for confusion matrix
    y_pred = prediction.flatten()
    y_act = y.flatten()
    return y_pred, y_act

In [ ]:
y_pred, y_act = logistic_regression_gradient_descent_demo(y_tr, X_tr, 1000, 0.7)

In [ ]:
# Visualize the Confusion Matrix
# np.set_printoptions(suppress=True) # disable scientific numbers
y_act = pd.Series(y_act, name='Actual')
y_pred = pd.Series(y_pred, name='Predicted')
conf_mat = pd.crosstab(y_act, y_pred, margins=True)
# print(conf_mat)
# Normalized confusion matrix
conf_mat_norm = conf_mat / conf_mat.sum(axis=1)
# print(conf_mat_norm)
# print(df_confusion)
# sn.heatmap(conf_mat, annot=True)

In [ ]:
labels = [0,1,2,3,4,5,6,7,8,9]
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)
fig.colorbar(cax)
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)
# plt.title('Binary Confusion Matrix',fontsize=16)
plt.xlabel('Predict')
plt.ylabel('Actual')

for (i, j), z in np.ndenumerate(conf_mat):
    ax.text(j, i, '{:0}'.format(z), ha='center', va='center') # for default in integer
#     ax.text(j, i, '{:0.2%}'.format(z), ha='center', va='center') # for normalized in percentage
    
plt.show()

## Stochastic Gradient Descent (Multiclass)

In [ ]:
def stochastic_gradient_descent(y, tx, w, batch_size, max_iter, tau, alpha):
    """Stochastic gradient descent."""
    losses = []
    accuracies = []
    y_enc = onehotencoding(y)
    for n_iter in range(max_iter):
        for y_batch, tx_batch in batch_iter(y_enc, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            gradient = calculate_gradient(y_batch, tx_batch, w)
            gradient += alpha*w
            # calculate loss
            softm, loss, prediction = calculate_loss(y_enc, tx, w)
            loss += (alpha/2) * np.sum(w*w)
            # update w through the stochastic gradient update
            w -= tau * gradient
            accuracy = np.sum(prediction == y)/ y.shape[0]
            taccuracy = getAccuracy(X_tst, y_tst, w)
            losses.append(loss)
            accuracies.append(accuracy)
        print("SGD({bi}/{ti}): loss={l}, training accuracy={a}, test accuracy={ta}".format(
              bi=n_iter, ti=max_iter - 1, l=loss, a=accuracy, ta=taccuracy))
        # visualization
        plt.plot(accuracies)
    return loss, w, accuracy, prediction

In [ ]:
def logistic_regression_stochastic_gradient_descent_demo(y, x, max_iter, tau):
    # init parameters
    max_iter = max_iter
    threshold = 1e-8
    tau = tau #change to see the diff
    alpha = 1e-7
    batch_size = 1

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 10))

    # Start SGD
    start_time = datetime.datetime.now()
    
    # start the logistic regression
    # get loss and update w.
    loss, w, accuracy, prediction = stochastic_gradient_descent(y, tx, w, batch_size, max_iter, tau, alpha)
    #taccuracy = getAccuracy(X_tst, y_tst, w)
    # log info
    end_time = datetime.datetime.now()
    exection_time = (end_time - start_time).total_seconds()
    # Print result
    print("SGD: execution time={t:.3f} seconds".format(t=exection_time))
    
    # return for confusion matrix
    y_pred = prediction.flatten()
    y_act = y.flatten()
    return y_pred, y_act

In [ ]:
y_pred, y_act = logistic_regression_stochastic_gradient_descent_demo(y_tr, X_tr, 1000, 0.5)

# Load fashion MNIST dataset
This dataset is taken from https://github.com/zalandoresearch/fashion-mnist

In [ ]:
fashion_images_tr, fashion_labels_tr = load_mnist('FashionMNIST/')
fashion_images_tst, fashion_labels_tst = load_mnist('FashionMNIST/', 't10k')

X_tr, y_tr = fashion_images_tr, fashion_labels_tr
X_tst, y_tst = fashion_images_tst, fashion_labels_tst


In [ ]:
# show some images
plt.figure(figsize=(9,9))
example_images = np.r_[X_tr[:12000:600], X_tr[13000:30600:600], X_tr[30600:60000:590]]
plot_images(example_images, images_per_row=10)
# save_fig("more_images_plot")
plt.show()

In [ ]:
# plot images for each label
X_0 = X_tr[(y_tr == 0)]
X_1 = X_tr[(y_tr == 1)]
X_2 = X_tr[(y_tr == 2)]
X_3 = X_tr[(y_tr == 3)]
X_4 = X_tr[(y_tr == 4)]
X_5 = X_tr[(y_tr == 5)]
X_6 = X_tr[(y_tr == 6)]
X_7 = X_tr[(y_tr == 7)]
X_8 = X_tr[(y_tr == 8)]
X_9 = X_tr[(y_tr == 9)]

plt.figure(figsize=(24,24))
plt.subplot(521); plot_images(X_0[:25], images_per_row=5)
plt.subplot(522); plot_images(X_1[:25], images_per_row=5)
plt.subplot(523); plot_images(X_2[:25], images_per_row=5)
plt.subplot(524); plot_images(X_3[:25], images_per_row=5)
plt.subplot(525); plot_images(X_4[:25], images_per_row=5)
plt.subplot(526); plot_images(X_5[:25], images_per_row=5)
plt.subplot(527); plot_images(X_6[:25], images_per_row=5)
plt.subplot(528); plot_images(X_7[:25], images_per_row=5)
plt.subplot(529); plot_images(X_8[:25], images_per_row=5)
plt.subplot(5,2,10); plot_images(X_9[:25], images_per_row=5)
# save_fig("images_for_each_label")
plt.show()

print(X_0.shape)
print(X_1.shape)

## Softmax Regression

Compute your cost by negative log likelihood.

In [ ]:
def logistic_regression_penalized_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 1000
    threshold = 1e-8
    tau = 1e-6 #change to see the diff
    alpha = 1e-7
    losses = []
    accuracies = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 10))

    # Start GD
    start_time = datetime.datetime.now()
    
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w, accuracy = learning_by_gradient_descent(y, tx, w, tau, alpha)
        taccuracy = getAccuracy(X_tst, y_tst, w)
        # log info
        if iter % 10 == 0:
            print("Current iteration={i}, loss={l}, training accuracy={a}, test accuracy={ta}"
                  .format(i=iter, l=loss, a=accuracy, ta=taccuracy))
        # converge criterion
        losses.append(loss)
        accuracies.append(accuracy)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # Print result
    end_time = datetime.datetime.now()
    exection_time = (end_time - start_time).total_seconds()
    print("GD: execution time={t:.3f} seconds".format(t=exection_time))
    # visualization
    #plt.plot(losses)
    plt.plot(accuracies)
    #visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_gradient_descent")
    #print("loss={l}".format(l=calculate_loss(onehotencoding(y), tx, w)))

In [ ]:
logistic_regression_penalized_gradient_descent_demo(y_tr, X_tr)

## Stochastic Gradient Descent

In [ ]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

In [ ]:
def stochastic_gradient_descent(y, tx, w, batch_size, max_iter, tau, alpha):
    """Stochastic gradient descent."""
    losses = []
    accuracies = []
    y_enc = onehotencoding(y)
    for n_iter in range(max_iter):
        for y_batch, tx_batch in batch_iter(y_enc, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            gradient = calculate_gradient(y_batch, tx_batch, w)
            gradient += alpha*w
            # calculate loss
            softm, loss, prediction = calculate_loss(y_enc, tx, w)
            loss += (alpha/2) * np.sum(w*w)
            # update w through the stochastic gradient update
            w -= tau * gradient
            accuracy = np.sum(prediction == y)/ y.shape[0]
            taccuracy = getAccuracy(X_tst, y_tst, w)
            losses.append(loss)
        print("SGD({bi}/{ti}): loss={l}, training accuracy={a}, test accuracy={ta}".format(
              bi=n_iter, ti=max_iter - 1, l=loss, a=accuracy, ta=taccuracy))
    return loss, w, accuracy

In [ ]:
def logistic_regression_stochastic_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 1000
    threshold = 1e-8
    tau = 1e-6 #change to see the diff
    alpha = 1e-7
    batch_size = 1

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 10))

    # Start SGD
    start_time = datetime.datetime.now()
    
    # start the logistic regression
    # get loss and update w.
    loss, w, accuracy = stochastic_gradient_descent(y, tx, w, batch_size, max_iter, tau, alpha)
    #taccuracy = getAccuracy(X_tst, y_tst, w)
    # log info
    # Print result
    end_time = datetime.datetime.now()
    exection_time = (end_time - start_time).total_seconds()
    print("SGD: execution time={t:.3f} seconds".format(t=exection_time))
    # visualization
    plt.plot(accuracies)
    #visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_gradient_descent")
    #print("loss={l}".format(l=calculate_loss(onehotencoding(y), tx, w)))

In [ ]:
logistic_regression_stochastic_gradient_descent_demo(y_tr, X_tr)

In [ ]:
#softmax newton 
#grid search